In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [8]:
data = pd.read_csv('player_stat_.csv')
data = data.drop(['player'], axis=1)
data

,G,GS,CG,SHO,W,L,SV,HLD,IP,BF,...,BB%,AVG,WHIP,BABIP,LOB%,ERA,FIP,LL,PF,WAR
0,27,27,1,0,9,9,0,0,182.2,734,...,0.063,0.232,1.090,0.320,0.780,2.66,2.41,0.50,1.0250,3.8
1,28,28,10,6,18,6,0,0,232.0,885,...,0.041,0.190,0.828,0.280,0.817,1.44,1.46,0.75,0.9870,4.7
2,28,28,0,0,12,7,0,0,158.1,674,...,0.070,0.250,1.260,0.309,0.717,4.09,4.40,0.50,0.9710,2.0
3,31,30,0,0,17,6,0,0,190.1,786,...,0.048,0.270,1.240,0.338,0.792,2.51,2.77,0.50,0.9710,0.6
4,30,30,0,0,20,3,0,0,194.2,773,...,0.038,0.226,1.000,0.285,0.787,2.50,2.87,0.50,0.9380,0.8
5,28,27,8,2,24,0,1,0,212.0,822,...,0.039,0.218,0.943,0.278,0.863,1.27,2.35,0.75,1.0010,2.9
6,29,29,5,0,15,8,0,0,206.3,821,...,0.050,0.222,1.013,0.282,0.798,2.09,2.38,0.75,0.9950,2.9
7,21,10,0,1,10,4,0,1,140.0,548,...,0.082,0.184,0.957,0.277,0.799,1.86,2.28,0.75,0.9870,1.0
8,23,23,1,0,14,4,0,0,163.7,654,...,0.069,0.210,1.033,0.256,0.757,3.08,3.52,0.75,1.0450,0.2
9,58,0,0,0,3,7,29,8,57.3,240,...,0.067,0.266,1.273,0.321,0.821,2.67,3.58,0.75,0.9870,0.3


In [2]:
#features deletion
data = pd.read_csv('player_stat_feature_deletion.csv')
data = data.drop(['player'], axis=1)

In [3]:
# pd.set_option('display.max_columns', 500)
# data = pd.read_csv('player_stat_.csv')
# data

In [3]:
data_norm = (data - data.min()) / (data.max() - data.min())
data_norm = data_norm.to_numpy()
data_norm

array([[0.72978839, 0.78304598, 0.72560976, 0.64705882, 0.71179039,
        0.95555556, 0.71830986, 0.72307692, 0.80821918, 0.5625    ,
        0.7480315 , 0.59259259, 0.55813953, 0.58876404, 0.7804878 ,
        0.375     , 0.4929078 , 0.32312925, 0.        , 0.52951917,
        0.82352941],
       [1.        , 1.        , 0.74390244, 0.23529412, 1.        ,
        0.46666667, 0.49295775, 0.46153846, 0.89041096, 0.14423077,
        0.95275591, 0.18518519, 0.06976744, 0.        , 0.29268293,
        0.5952381 , 0.06028369, 0.        , 1.        , 0.29823494,
        1.        ],
       [0.59902333, 0.69683908, 0.7195122 , 1.        , 0.49781659,
        0.71111111, 1.        , 1.        , 0.53424658, 0.75480769,
        0.37795276, 0.72222222, 0.76744186, 0.97078652, 0.64634146,
        0.        , 1.        , 1.        , 0.        , 0.2008521 ,
        0.47058824],
       [0.77265328, 0.85775862, 1.        , 0.70588235, 0.58078603,
        0.51111111, 0.8028169 , 0.70769231, 0.3881278

In [4]:
training_set_1 = torch.FloatTensor(data_norm[0:8])
test_set_1 = torch.FloatTensor(data_norm[8:14])
# test_set_1 = torch[8:14]
# training_set_1

In [5]:
nb_player = len(training_set_1)
nb_stats = data_norm.shape[1]

In [6]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(21, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 16)
        self.fc4 = nn.Linear(16, 21)
#         self.fc1 = nn.Linear(29, 10)
#         self.fc2 = nn.Linear(10, 1)
#         self.fc3 = nn.Linear(1, 10)
#         self.fc4 = nn.Linear(10, 29)
        self.activation = nn.Sigmoid()
#         self.activation = nn.ReLU()
        
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

In [7]:
nb_epoch = 200 #200
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_player in range(nb_player):
        input = Variable(training_set_1[id_player]).unsqueeze(0)
        target = input.clone()
        #Select only rating related columns to compute loss
        target_ratings = target[:, :nb_stats]
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            output_ratings = output[:, :nb_stats]
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output_ratings, target_ratings)
            mean_corrector = nb_stats/float(torch.sum(target.data > 0) + 1e-10)
#             print(mean_corrector)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
    
# Testing the SAE
test_loss = 0
s = 0.
for id_player in range(len(test_set_1)):
    input = Variable(test_set_1[id_player]).unsqueeze(0)
    target = Variable(test_set_1[id_player]).unsqueeze(0)
    target_ratings = target[:, :nb_stats]
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        output_ratings = output[:, :nb_stats]
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output_ratings, target_ratings)
        mean_corrector = nb_stats/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

epoch: 1 loss: tensor(0.4512)
epoch: 2 loss: tensor(0.2701)
epoch: 3 loss: tensor(0.2428)
epoch: 4 loss: tensor(0.2414)
epoch: 5 loss: tensor(0.2396)
epoch: 6 loss: tensor(0.2379)
epoch: 7 loss: tensor(0.2368)
epoch: 8 loss: tensor(0.2359)
epoch: 9 loss: tensor(0.2353)
epoch: 10 loss: tensor(0.2349)
epoch: 11 loss: tensor(0.2346)
epoch: 12 loss: tensor(0.2343)
epoch: 13 loss: tensor(0.2341)
epoch: 14 loss: tensor(0.2339)
epoch: 15 loss: tensor(0.2337)
epoch: 16 loss: tensor(0.2336)
epoch: 17 loss: tensor(0.2335)
epoch: 18 loss: tensor(0.2333)
epoch: 19 loss: tensor(0.2332)
epoch: 20 loss: tensor(0.2331)
epoch: 21 loss: tensor(0.2331)
epoch: 22 loss: tensor(0.2330)
epoch: 23 loss: tensor(0.2330)
epoch: 24 loss: tensor(0.2329)
epoch: 25 loss: tensor(0.2329)
epoch: 26 loss: tensor(0.2329)
epoch: 27 loss: tensor(0.2328)
epoch: 28 loss: tensor(0.2327)
epoch: 29 loss: tensor(0.2327)
epoch: 30 loss: tensor(0.2327)
epoch: 31 loss: tensor(0.2326)
epoch: 32 loss: tensor(0.2327)
epoch: 33 loss: t

In [9]:
w = list(sae.parameters())
w

[Parameter containing:
 tensor([[-5.0258e-03, -2.5123e-23, -5.0109e-03, -4.9885e-03,  5.0464e-03,
          -3.3497e-23, -8.6881e-44, -5.0308e-03, -3.7745e-23, -3.8875e-23,
          -2.5276e-23, -1.3300e-23, -4.9899e-03, -5.0077e-03, -2.7600e-23,
          -2.5509e-23, -7.5368e-23,  4.9681e-03, -7.5368e-23, -7.1181e-23,
           4.9792e-03, -5.0142e-03, -5.0130e-03, -5.0350e-03, -1.5769e-23,
          -2.1021e-23, -5.0031e-03, -5.0307e-03,  5.0364e-03],
         [ 1.3312e-43, -1.1070e-43,  4.9766e-03,  1.2191e-43, -8.4078e-45,
          -5.7453e-44, -4.9352e-03,  7.9874e-44,  5.8855e-44,  9.1084e-44,
           4.9702e-03, -1.0930e-43,  5.0257e-03,  6.1657e-44,  3.2230e-44,
          -1.0650e-43,  4.9765e-03,  4.9686e-03,  9.8091e-44, -2.8026e-45,
          -5.0116e-03, -4.9618e-03,  2.9427e-44, -6.8664e-44, -5.8855e-44,
          -5.0188e-03, -8.5479e-44, -5.0014e-03,  5.0313e-03],
         [-4.9726e-03, -2.1259e-16, -2.9427e-44, -1.0629e-16, -5.0207e-03,
          -4.9761e-03,  4.

In [10]:
for name, param in sae.named_parameters():
    if param.requires_grad:
        print (name, param.data)

fc1.weight tensor([[-5.0258e-03, -2.5123e-23, -5.0109e-03, -4.9885e-03,  5.0464e-03,
         -3.3497e-23, -8.6881e-44, -5.0308e-03, -3.7745e-23, -3.8875e-23,
         -2.5276e-23, -1.3300e-23, -4.9899e-03, -5.0077e-03, -2.7600e-23,
         -2.5509e-23, -7.5368e-23,  4.9681e-03, -7.5368e-23, -7.1181e-23,
          4.9792e-03, -5.0142e-03, -5.0130e-03, -5.0350e-03, -1.5769e-23,
         -2.1021e-23, -5.0031e-03, -5.0307e-03,  5.0364e-03],
        [ 1.3312e-43, -1.1070e-43,  4.9766e-03,  1.2191e-43, -8.4078e-45,
         -5.7453e-44, -4.9352e-03,  7.9874e-44,  5.8855e-44,  9.1084e-44,
          4.9702e-03, -1.0930e-43,  5.0257e-03,  6.1657e-44,  3.2230e-44,
         -1.0650e-43,  4.9765e-03,  4.9686e-03,  9.8091e-44, -2.8026e-45,
         -5.0116e-03, -4.9618e-03,  2.9427e-44, -6.8664e-44, -5.8855e-44,
         -5.0188e-03, -8.5479e-44, -5.0014e-03,  5.0313e-03],
        [-4.9726e-03, -2.1259e-16, -2.9427e-44, -1.0629e-16, -5.0207e-03,
         -4.9761e-03,  4.9952e-03, -7.9720e-17, -5.

In [11]:
PATH = './model/baseball.pth'
torch.save(sae.state_dict(), PATH)

In [12]:
outputs = sae(test_set_1)
outputs

tensor([[0.2517, 0.8738, 0.6103, 0.4903, 0.6355, 0.6678, 0.0486, 0.1365, 0.7690,
         0.8129, 0.7506, 0.4995, 0.6237, 0.5540, 0.6300, 0.6125, 0.5408, 0.3922,
         0.5113, 0.4117, 0.5376, 0.5421, 0.4815, 0.5357, 0.4203, 0.4463, 1.0091,
         0.3172, 0.5213],
        [0.2517, 0.8738, 0.6103, 0.4903, 0.6355, 0.6678, 0.0486, 0.1365, 0.7690,
         0.8129, 0.7506, 0.4995, 0.6237, 0.5540, 0.6300, 0.6125, 0.5408, 0.3922,
         0.5113, 0.4117, 0.5376, 0.5421, 0.4815, 0.5357, 0.4203, 0.4463, 1.0091,
         0.3172, 0.5213],
        [0.2517, 0.8738, 0.6103, 0.4903, 0.6355, 0.6678, 0.0486, 0.1365, 0.7690,
         0.8129, 0.7506, 0.4995, 0.6237, 0.5540, 0.6300, 0.6125, 0.5408, 0.3922,
         0.5113, 0.4117, 0.5376, 0.5421, 0.4815, 0.5357, 0.4203, 0.4463, 1.0091,
         0.3172, 0.5213],
        [0.2517, 0.8738, 0.6103, 0.4903, 0.6355, 0.6678, 0.0486, 0.1365, 0.7690,
         0.8129, 0.7506, 0.4995, 0.6237, 0.5540, 0.6300, 0.6125, 0.5408, 0.3922,
         0.5113, 0.4117, 0.5376

In [13]:
test_set_1

tensor([[0.1463, 0.7667, 0.1000, 0.0000, 0.5455, 0.4444, 0.0000, 0.0000, 0.6294,
         0.6681, 0.5488, 0.8824, 0.4629, 0.6667, 0.7324, 0.7538, 0.3653, 0.6587,
         0.3386, 0.7037, 0.3023, 0.4607, 0.0000, 0.2381, 0.6418, 0.7007, 1.0000,
         0.6512, 0.1176],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0455, 0.7778, 1.0000, 1.0000, 0.0521,
         0.0733, 0.1402, 0.2353, 0.0000, 0.0222, 0.1690, 0.1538, 0.1301, 0.6779,
         0.0394, 0.6667, 0.9535, 1.0000, 0.7927, 0.6190, 0.4965, 0.7211, 1.0000,
         0.2982, 0.1373],
        [0.2195, 0.8667, 0.0000, 0.0000, 0.5909, 0.4444, 0.0000, 0.0000, 0.6636,
         0.7414, 0.6280, 0.4118, 0.6157, 1.0000, 0.7465, 0.7385, 0.7169, 1.0000,
         0.5984, 1.0000, 0.4419, 0.7438, 0.6098, 0.2262, 0.5816, 0.5476, 1.0000,
         1.0000, 0.0000],
        [0.0000, 0.5667, 0.2000, 0.1667, 0.1818, 0.7778, 0.0000, 0.0000, 0.3869,
         0.4052, 0.4390, 0.2941, 0.1878, 0.0889, 0.3803, 0.3846, 0.0000, 0.1635,
         0.0000, 0.1667, 0.6977

In [14]:
result = criterion(test_set_1,outputs)
result

tensor(0.1387, grad_fn=<MeanBackward0>)